In [49]:
import requests
import lxml.html as lh
import pandas as pd

In [50]:
#example url
url='http://www.plusliga.pl/statsPlayers/id/304.html'

In [51]:
def scrap_table_from_url(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    
    return tr_elements

In [52]:
tr_elements = scrap_table_from_url(url)

In [53]:
#Check the length of all the rows
#we have two tables on the webpage
#two rows are shorter (7 elements), because both tables on webpage have two header lines
[len(T) for T in tr_elements[0:]]

[7,
 19,
 19,
 7,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19]

In [54]:
def create_headers(tr_elements):
    #create empty list
    col1=[]
    i=0
    
    #store headers from first row in table
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col1.append(name)
    
    #create empty list
    col2=[]
    j=0
    #store headers from second row in table
    for t in tr_elements[1]:
        j+=1
        name=t.text_content()
        col2.append(name)
    
    #create one header names for all table
    col2[3]=col1[3] + '_' + col2[3]
    col2[4]=col1[3] + '_' + col2[4]
    col2[5]=col1[3] + '_' + col2[5]
    col2[6]=col1[3] + '_' + col2[6]
    col2[7]=col1[4] + '_' + col2[7]
    col2[8]=col1[4] + '_' + col2[8]
    col2[9]=col1[4] + '_' + col2[9]
    col2[10]=col1[4] + '_' + col2[10]
    col2[11]=col1[4] + '_' + col2[11]
    col2[12]=col1[5] + '_' + col2[12]
    col2[13]=col1[5] + '_' + col2[13]
    col2[14]=col1[5] + '_' + col2[14]
    col2[15]=col1[5] + '_' + col2[15]
    col2[16]=col1[5] + '_' + col2[16]
    col2[17]=col1[6] + '_' + col2[17]
    col2[18]=col1[6] + '_' + col2[18]
    
    return col2

In [55]:
headers = create_headers(tr_elements)
headers

['Rozegrane mecze',
 'Sety',
 'Punkty',
 'Zagrywka_Suma',
 'Zagrywka_As',
 'Zagrywka_Błąd',
 'Zagrywka_Asy na set',
 'Przyjęcie_Suma',
 'Przyjęcie_Błąd',
 'Przyjęcie_Neg',
 'Przyjęcie_Perf',
 'Przyjęcie_% perf',
 'Atak_Suma',
 'Atak_Błąd',
 'Atak_Blok',
 'Atak_Perf',
 'Atak_% perf',
 'Blok_Pkt',
 'Blok_Pkt na set']

In [56]:
def make_space_in_headers(headers):
    #change list to tuple
    headers_tuple = tuple(headers)
    #create empty list
    col=[]
    #add empty lists to tuple
    for t in range(0,len(headers_tuple)):
        col.append((headers_tuple[t],[]))
    return col

In [57]:
col = make_space_in_headers(headers)

In [58]:
#Our data are stored on the second table, which is from 5th row onwards without last two rows
def fill_data(tr_elements,col):
    row_width = len(tr_elements[4])
    
    for j in range(5,len(tr_elements)-2):
        #T is our j'th row
        T=tr_elements[j]
    
        #If row is not of size row_width (19), the //tr data is not from our table 
        if len(T)!=row_width:
            break
    
        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
    return col

In [59]:
col = fill_data(tr_elements,col)

In [60]:
#checking the length of each column
[len(C) for (title,C) in col]

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]

In [61]:
def create_dataframe(col):
    #creating dataframe from dict
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [62]:
create_dataframe(col)

,Rozegrane mecze,Sety,Punkty,Zagrywka_Suma,Zagrywka_As,Zagrywka_Błąd,Zagrywka_Asy na set,Przyjęcie_Suma,Przyjęcie_Błąd,Przyjęcie_Neg,Przyjęcie_Perf,Przyjęcie_% perf,Atak_Suma,Atak_Błąd,Atak_Blok,Atak_Perf,Atak_% perf,Blok_Pkt,Blok_Pkt na set
0,Trefl Gdańsk -PGE Skra Bełchatów,4,22,20,1,3,"0,25",0,0,0,0,"0,00",39,2,3,20,"51,28",1,"0,25"
1,Trefl Gdańsk -Cerrad Enea Czarni Radom,4,19,17,3,0,"0,75",0,0,0,0,"0,00",28,0,3,15,"53,57",1,"0,25"
2,Trefl Gdańsk -GKS Katowice,5,17,16,1,5,"0,20",0,0,0,0,"0,00",38,4,3,15,"39,47",1,"0,20"
3,Trefl Gdańsk -Ślepsk Malow Suwałki,5,29,13,1,4,"0,20",0,0,0,0,"0,00",44,2,7,27,"61,36",1,"0,20"
4,Trefl Gdańsk -Indykpol AZS Olsztyn,4,23,20,2,6,"0,50",1,1,0,0,"0,00",41,2,4,19,"46,34",2,"0,50"
5,Trefl Gdańsk -Cuprum Lubin,3,18,10,1,1,"0,33",1,0,0,0,"0,00",27,2,1,12,"44,44",5,"1,67"
6,Trefl Gdańsk -BKS Visła Bydgoszcz,5,18,15,0,2,"0,00",2,1,1,0,"0,00",32,5,3,14,"43,75",4,"0,80"
7,Trefl Gdańsk -MKS Będzin,4,19,22,2,4,"0,50",1,0,1,0,"0,00",36,2,2,14,"38,89",3,"0,75"
8,Trefl Gdańsk -Grupa Azoty ZAKSA Kędzierzyn-Koźle,4,22,15,2,7,"0,50",1,0,1,0,"0,00",37,1,2,19,"51,35",1,"0,25"
9,Trefl Gdańsk -VERVA Warszawa ORLEN Paliwa,3,16,13,2,4,"0,67",0,0,0,0,"0,00",32,3,3,14,"43,75",0,"0,00"


In [63]:
#scrap name of the player 
def scrap_name(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//h1')
    name = tr_elements[0].text_content()
    
    return name

In [64]:
def scrap_table_to_df_with_player_name(url):
    headers = create_headers(scrap_table_from_url(url))
    col = make_space_in_headers(headers)
    col1 = fill_data(scrap_table_from_url(url), col)
    df = create_dataframe(col1)
    df['Gracz'] = scrap_name(url)
    return df

In [65]:
df = scrap_table_to_df_with_player_name(url)

In [66]:
df.head(25)

,Rozegrane mecze,Sety,Punkty,Zagrywka_Suma,Zagrywka_As,Zagrywka_Błąd,Zagrywka_Asy na set,Przyjęcie_Suma,Przyjęcie_Błąd,Przyjęcie_Neg,Przyjęcie_Perf,Przyjęcie_% perf,Atak_Suma,Atak_Błąd,Atak_Blok,Atak_Perf,Atak_% perf,Blok_Pkt,Blok_Pkt na set,Gracz
0,Trefl Gdańsk -PGE Skra Bełchatów,4,22,20,1,3,"0,25",0,0,0,0,"0,00",39,2,3,20,"51,28",1,"0,25",Bartosz Filipiak
1,Trefl Gdańsk -Cerrad Enea Czarni Radom,4,19,17,3,0,"0,75",0,0,0,0,"0,00",28,0,3,15,"53,57",1,"0,25",Bartosz Filipiak
2,Trefl Gdańsk -GKS Katowice,5,17,16,1,5,"0,20",0,0,0,0,"0,00",38,4,3,15,"39,47",1,"0,20",Bartosz Filipiak
3,Trefl Gdańsk -Ślepsk Malow Suwałki,5,29,13,1,4,"0,20",0,0,0,0,"0,00",44,2,7,27,"61,36",1,"0,20",Bartosz Filipiak
4,Trefl Gdańsk -Indykpol AZS Olsztyn,4,23,20,2,6,"0,50",1,1,0,0,"0,00",41,2,4,19,"46,34",2,"0,50",Bartosz Filipiak
5,Trefl Gdańsk -Cuprum Lubin,3,18,10,1,1,"0,33",1,0,0,0,"0,00",27,2,1,12,"44,44",5,"1,67",Bartosz Filipiak
6,Trefl Gdańsk -BKS Visła Bydgoszcz,5,18,15,0,2,"0,00",2,1,1,0,"0,00",32,5,3,14,"43,75",4,"0,80",Bartosz Filipiak
7,Trefl Gdańsk -MKS Będzin,4,19,22,2,4,"0,50",1,0,1,0,"0,00",36,2,2,14,"38,89",3,"0,75",Bartosz Filipiak
8,Trefl Gdańsk -Grupa Azoty ZAKSA Kędzierzyn-Koźle,4,22,15,2,7,"0,50",1,0,1,0,"0,00",37,1,2,19,"51,35",1,"0,25",Bartosz Filipiak
9,Trefl Gdańsk -VERVA Warszawa ORLEN Paliwa,3,16,13,2,4,"0,67",0,0,0,0,"0,00",32,3,3,14,"43,75",0,"0,00",Bartosz Filipiak
